In [36]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout


In [37]:
user_profiles = pd.read_csv('aggregated_user_profiles.csv')

songs = pd.read_csv('30k songs data\spotify_songs.csv')

In [38]:
user_profiles

,userid,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Genre 1,Genre 2,Genre 3
0,00055176fea33f6e027cd3302289378b,Green Day,0.528600,0.780033,4.800000,-5.466067,0.933333,0.067063,0.097408,0.002713,0.279387,0.601667,119.757400,217063.966667,4.0,modern rock,modern rock,NaN
1,0007f3dd09c91198371454c608d47f22,Ed Sheeran,0.503923,0.552474,4.410256,-8.301244,0.769231,0.043521,0.359119,0.033772,0.131090,0.336121,107.436705,255803.807692,4.0,pop,singer-songwriter pop,NaN
2,000b0f32b5739f052b9d40fcc5c41079,First Aid Kit,0.514333,0.678556,5.111111,-6.382000,0.888889,0.040611,0.226000,0.002485,0.150211,0.393889,114.546222,240551.111111,4.0,swedish americana,swedish country,NaN
3,000c11a16c89aa4b14b328080f5954ee,Drake,0.604992,0.648232,4.896000,-7.674204,0.660000,0.058918,0.199622,0.077143,0.175239,0.519824,119.436044,249030.780000,4.0,alternative dance,pop,NaN
4,00123e0f544dee3ab006aa7f1e5725a7,Tom Waits,0.545406,0.635927,5.483384,-8.870202,0.703927,0.058784,0.267978,0.145345,0.204341,0.514726,116.258459,244649.264350,4.0,album rock,singer-songwriter,classic rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14440,ffe11226cdea81a2db9262c0ec7f5d71,Nickelback,0.456223,0.766661,5.589286,-6.639098,0.812500,0.049376,0.109372,0.060770,0.200111,0.494063,125.454250,256360.821429,4.0,alternative metal,hard rock,post-grunge
14441,ffe32d5412269f3041c58cbf0dde3306,Rilo Kiley,0.624500,0.575808,4.653846,-8.967038,0.846154,0.052046,0.290518,0.036739,0.126188,0.531654,115.183269,237432.846154,4.0,classic soul,funktronica,indie pop
14442,fff60baf392613ed33f745b89a9b38f7,Nas,0.586826,0.793535,5.860465,-5.969314,0.639535,0.127047,0.098213,0.207107,0.241108,0.569244,113.582686,231586.279070,4.0,conscious hip hop,east coast hip hop,gangster rap
14443,fff616055993498d6127f3f467cf9f2b,Of Monsters and Men,0.563826,0.642680,4.941714,-7.420387,0.715429,0.054694,0.237019,0.080279,0.168059,0.433974,120.363894,237590.517714,4.0,chamber pop,modern rock,NaN


In [39]:
user_profiles.columns

Index(['userid', 'artist', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature', 'Genre 1',
       'Genre 2', 'Genre 3'],
      dtype='object')

In [40]:
# drop columns Genre 2 and Genre 3 and rename Genre 1 to Genre
user_profiles = user_profiles.drop(columns=['Genre 2', 'Genre 3'])
user_profiles = user_profiles.rename(columns={'Genre 1': 'Genre'})

user_profiles

,userid,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Genre
0,00055176fea33f6e027cd3302289378b,Green Day,0.528600,0.780033,4.800000,-5.466067,0.933333,0.067063,0.097408,0.002713,0.279387,0.601667,119.757400,217063.966667,4.0,modern rock
1,0007f3dd09c91198371454c608d47f22,Ed Sheeran,0.503923,0.552474,4.410256,-8.301244,0.769231,0.043521,0.359119,0.033772,0.131090,0.336121,107.436705,255803.807692,4.0,pop
2,000b0f32b5739f052b9d40fcc5c41079,First Aid Kit,0.514333,0.678556,5.111111,-6.382000,0.888889,0.040611,0.226000,0.002485,0.150211,0.393889,114.546222,240551.111111,4.0,swedish americana
3,000c11a16c89aa4b14b328080f5954ee,Drake,0.604992,0.648232,4.896000,-7.674204,0.660000,0.058918,0.199622,0.077143,0.175239,0.519824,119.436044,249030.780000,4.0,alternative dance
4,00123e0f544dee3ab006aa7f1e5725a7,Tom Waits,0.545406,0.635927,5.483384,-8.870202,0.703927,0.058784,0.267978,0.145345,0.204341,0.514726,116.258459,244649.264350,4.0,album rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14440,ffe11226cdea81a2db9262c0ec7f5d71,Nickelback,0.456223,0.766661,5.589286,-6.639098,0.812500,0.049376,0.109372,0.060770,0.200111,0.494063,125.454250,256360.821429,4.0,alternative metal
14441,ffe32d5412269f3041c58cbf0dde3306,Rilo Kiley,0.624500,0.575808,4.653846,-8.967038,0.846154,0.052046,0.290518,0.036739,0.126188,0.531654,115.183269,237432.846154,4.0,classic soul
14442,fff60baf392613ed33f745b89a9b38f7,Nas,0.586826,0.793535,5.860465,-5.969314,0.639535,0.127047,0.098213,0.207107,0.241108,0.569244,113.582686,231586.279070,4.0,conscious hip hop
14443,fff616055993498d6127f3f467cf9f2b,Of Monsters and Men,0.563826,0.642680,4.941714,-7.420387,0.715429,0.054694,0.237019,0.080279,0.168059,0.433974,120.363894,237590.517714,4.0,chamber pop


In [41]:
songs.columns

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms'],
      dtype='object')

## Data Preprocessing

In [42]:
# Normalize numerical features
numerical_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
scaler = MinMaxScaler()
user_profiles[numerical_features] = scaler.fit_transform(user_profiles[numerical_features])
songs[numerical_features] = scaler.transform(songs[numerical_features])

# One-hot encode categorical features for genres
encoder = OneHotEncoder()
user_genres_encoded = encoder.fit_transform(user_profiles[['Genre']]).toarray()
songs_genres_encoded = encoder.transform(songs[['playlist_genre', 'playlist_subgenre']]).toarray()

# Concatenate normalized and encoded features back to the original dataframes
user_profiles = pd.concat([user_profiles.drop('Genre', axis=1), pd.DataFrame(user_genres_encoded, columns=encoder.get_feature_names_out())], axis=1)
songs = pd.concat([songs.drop(['playlist_genre', 'playlist_subgenre'], axis=1), pd.DataFrame(songs_genres_encoded, columns=encoder.get_feature_names_out())], axis=1)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- playlist_genre
- playlist_subgenre
Feature names seen at fit time, yet now missing:
- Genre


# Creating Training Data

For the training of your neural network-based recommendation system, we need to generate training samples. This involves creating pairs of user profiles and songs, with labels indicating whether a user would like a particular song. The labels can be based on a heuristic, such as a similarity score between the user’s preferences and the song’s characteristics, or it could be more sophisticated based on user interactions, if available. Since we don't have explicit like/dislike data, let’s proceed with a heuristic-based approach.

The goal here is to generate a dataset where each instance consists of:

A user profile vector.
A song vector.
A binary label (0 or 1), where 1 might indicate that the song matches the user's preferences well based on a heuristic.
We'll define a simple heuristic for matching:

Calculate the cosine similarity between the numerical features of the user profile and the song.
Treat genres as matching if there's any overlap.
A song is considered a good match (label=1) if it has a high similarity score and matching genres

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [ ]:
def batch_generator(users, songs, batch_size=100, threshold=0.7):
    # Prepare indices for batches
    num_users = users.shape[0]
    num_songs = songs.shape[0]

    # Generate batches
    for user_start in range(0, num_users, batch_size):
        user_end = min(user_start + batch_size, num_users)
        user_batch = users.iloc[user_start:user_end]

        user_inputs = []
        song_inputs = []
        labels = []

        # Process each user in the batch
        for _, user in user_batch.iterrows():
            user_vector = user[numerical_features].values.reshape(1, -1)
            user_genres = user[encoder.get_feature_names_out()].values

            # Compute similarities with all songs
            for idx in range(num_songs):
                song = songs.iloc[idx]
                song_vector = song[numerical_features].values.reshape(1, -1)
                song_genres = song[encoder.get_feature_names_out()].values

                similarity = cosine_similarity(user_vector, song_vector).flatten()[0]
                genre_overlap = (user_genres * song_genres).sum() > 0
                label = int(similarity > threshold and genre_overlap)

                labels.append(label)
                user_inputs.append(user.values)
                song_inputs.append(song.values)

        yield np.array(user_inputs), np.array(song_inputs), np.array(labels)

# Use this generator within a training loop


KeyError: "None of [Index(['Genre 1', 'Genre 2', 'Genre 3'], dtype='object')] are in the [columns]"

In [ ]:
X_user_train, X_user_test, X_song_train, X_song_test, y_train, y_test = train_test_split(
    np.hstack((user_inputs, song_inputs)), labels, test_size=0.2, random_state=42
)

